# Transfer learning pour la classificaion cats vs. dogs 

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from keras.optimizers import adam
from keras import applications
from keras import backend as K

Dans ce cahier, nous utiliserons des images étiquetées de chats et de chiens pour former un classifieur capable de les distinguer. Comme beaucoup d’autres dans le domaine de la vision par ordinateur, c’est une tâche que nous avons appris à maîtriser au cours des dernières années, principalement grâce aux réseaux de neurones à convolution.
Il y a eu un concours sur Kaggle pour cela en $2014$. Pierre Sermanet, élève de Yann LeCun, a pris la 1ère place, avec une impressionnante précision de 98,9% sur le set de test. Il explique brièvement comment il l'a fait dans cet article Google+:

"*Je viens de gagner le concours Dogs vs. Cats Kaggle, en utilisant la bibliothèque d'apprentissage en profondeur que j'avais écrite lors de ma thèse: OverFeat http://cilvr.nyu.edu/doku.php?id=code:start
Mon système a été pré-formé sur ImageNet (ensemble de données de classification ILSVRC12) et ensuite affiné sur les données de chats et de chiens. J'ai limité mon nombre de soumissions à 5 pour éviter le réglage des ensembles de tests et j'ai obtenu la 1ère place contre 215 autres équipes avec une précision de 98,9%.*"

Mais qu'est-ce que cela signifie exactement de pré-former un modèle? Eh bien, découvrons!

## Charger les data

Nous allons utiliser les données du concours Kaggle. Le jeu de données est assez volumineux: il existe 25 000 échantillons d’entraînement et 1 200 échantillons d’essai, tous de tailles différentes. Nous allons donc travailler avec seulement quelques-uns d'entre eux: nous prenons 3000 échantillons du kit de formation, et nous en utilisons 2000 pour la formation et 1000 pour la validation.

**WARNING**: telechargez le  dataset plus peti ici [here](https://www.dropbox.com/s/teiafmdpt49pddd/cats_and_dogs_small.zip?dl=0) puis un-zipez le dans un dossier `mldata`.

Chargeons les données - Keras a quelques bonnes routines pour cela. Nous commençons par charger une seule image, juste pour voir à quoi ça ressemble (essayez de changer le nom du fichier pour en voir d'autres). Notez que nous faisons des images en couleurs et que nous travaillons donc avec des tableaux à 3 dimensions.

In [ ]:
# Load image and transform it to a Numpy array
img = load_img("mldata/cats_and_dogs_small/train/cats/cat.2.jpg")
x = img_to_array(img)
print("array size:", x.shape)

# Show image
plt.imshow(x / 255.);

Ensuite, nous spécifions quelques paramètres de formation et créons un * générateur * pour les données, en utilisant un outil très pratique de Keras appelé `ImageDataGenerator`. Essentiellement, il parcourt les images du répertoire et les traite à notre intention.

Ci-dessous, nous redimensionnons chacune des images afin que chaque intensité de pixel passe de 0 à 1 au lieu de 0 à 255; et également ajouter du bruit à l'image pour que le classificateur devienne plus robuste, une technique connue sous le nom de *augmentation de données*.

In [ ]:
# Parameters
batch_size = 16
epochs = 20

n_train_samples = 2000
n_valid_samples = 1000
img_width, img_height = 150, 150

# Set up generator for training and validation images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)
train_generator = train_datagen.flow_from_directory("mldata/cats_and_dogs_small/train",
                                                    target_size=(img_height, img_width),
                                                    batch_size=batch_size,
                                                    class_mode="binary")

test_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = test_datagen.flow_from_directory("mldata/cats_and_dogs_small/valid",
                                                   target_size=(img_height, img_width),
                                                   batch_size=batch_size,
                                                   class_mode="binary")

## Entrainement d'un CNN de zero!

Enfin, nous allons écrire notre réseau de neurones en utilisant Keras et le former. Ci-dessous, nous utilisons un réseau de neurones avec 3 couches convolutives censées apprendre les caractéristiques pertinentes et 2 couches plus denses qui utiliseront les caractéristiques apprises pour classer l'image en tant que chat ou chien.

In [ ]:
# Specify network architecture: 3 conv. layers w/ ReLU activations + 2 dense layers
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation("sigmoid"))

model.compile(loss="binary_crossentropy",
              optimizer="rmsprop",
              metrics=["accuracy"])
model.summary()

Il nous faut maintenant faire l'entrainement, ce qui prend un peu de temps!

In [ ]:
# Fit our model
#model.fit_generator(
    #train_generator,
    #steps_per_epoch=n_train_samples // batch_size,
    #epochs=epochs,
    #validation_data=valid_generator,
    #validation_steps=n_valid_samples // batch_size)

#model.save_weights("mldata/cnn_catsvsdogs.h5")

In [ ]:
# (or if you don't want to wait you can't just load the weights below)
model.load_weights("mldata/cnn_catsvsdogs.h5")

Ouf, ça a pris longtemps (sauf si vous avez un GPU!). Si seulement nous pouvions utiliser des poids que nous avons déjà formés sur d'autres jeux de données ...

Voyons quelle est la précision obtenue sur l'ensemble de test:

In [ ]:
test_loss, test_acc = model.evaluate_generator(valid_generator, steps=n_valid_samples//batch_size)
print("accuracy on test set:", test_acc)

Pas mal - avec une telle précision, nous serions parmi la moitié supérieure du concours Kaggle (voir réf. 1).

## Untiliser un model pre-entrainé

La formation de notre CNN a pris beaucoup de temps. Et si nous le remplaions par un autre, formé dans un * jeu de données similaire *? Après tout, les caractéristiques importantes devraient être plus ou moins les mêmes, non?

Les gens ont utilisé beaucoup de ressources informatiques, formant des réseaux très profonds sur de vastes ensembles de données. Heureusement, ils ont mis leurs poids à disposition afin que nous puissions les utiliser! Ci-dessous, nous allons utiliser un réseau de neurones convolutifs à 16 couches formé sur le jeu de données Imagenet, connu sous le nom de [VGG-16] (http://www.robots.ox.ac.uk/%7Evgg/research/very_deep/).

![VGG16](vgg16.png)

Faisons-le par étapes: nous chargeons d'abord le réseau VGG16, sans la dernière couche de classificateur(`include_top=False`).

In [ ]:
# Load VGG16 weights
vgg16 = applications.VGG16(include_top=False, weights="imagenet")

Ensuite, nous créons les générateurs comme auparavant (maintenant sans aucune augmentation de données) et passons l'ensemble de nos échantillons via le réseau VGG16.

In [ ]:
# Parameters
datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = vgg16.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            print(f"{i * batch_size} samples generated for {directory}")
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

In [ ]:
# This makes take a while...
n_train_samples = 1000
n_valid_samples = 500

train_features, train_labels = extract_features("mldata/cats_and_dogs_small/train", n_train_samples)
valid_features, valid_labels = extract_features("mldata/cats_and_dogs_small/valid", n_valid_samples)
print(f"train_features: {train_features.shape}")
print(f"valid_features: {valid_features.shape}")

Essayons de comprendre ce que fait le réseau VGG16. Voyons d'abord à quoi ressemble la sortie.

In [ ]:
fig, axs = plt.subplots(8, 8, figsize=(7, 7))
for i in range(64):
    axs[i // 8, i % 8].imshow(train_features[0, :, :, i], cmap="gray")
    axs[i // 8, i % 8].get_xaxis().set_visible(False)
    axs[i // 8, i % 8].get_yaxis().set_visible(False)
fig.subplots_adjust(hspace=0.1)

Il consiste en 512 images de taille 4x4, mais les regarder ne nous révèle pas grand-chose ... Regardons la sortie des filtres un par un.

In [ ]:
# Create function that reads image and returns each layer output
input_img = vgg16.input
outputs = [layer.output for layer in vgg16.layers][1:]
functors = [K.function([input_img], [out]) for out in outputs]

# Input first image in the training set to this function
img_generator = datagen.flow_from_directory(
    "mldata/cats_and_dogs_small/train",
    target_size=(150, 150),
    batch_size=1,
    class_mode=None,
    shuffle=False
)
img_generator.reset()
img = img_generator.next()

layer_outputs = [func([img])[0] for func in functors]
for i in range(len(layer_outputs)):
    print("layer %d shape: %s" % (i, layer_outputs[i].shape))

Changez la valeur de `layer` ci-dessous, de 0 à 17.

In [ ]:
layer = 1

fig, axs = plt.subplots(8, 8, figsize=(14, 14))
for i in range(64):
    axs[i // 8, i % 8].imshow(layer_outputs[layer][0, :, :, i], cmap="gray")
    axs[i // 8, i % 8].get_xaxis().set_visible(False)
    axs[i // 8, i % 8].get_yaxis().set_visible(False)
fig.subplots_adjust(hspace=0.1)

Nous pouvons comprendre chaque couche d'un CNN comme effectuant plusieurs tâches de traitement d'image en parallèle - détection des contours, netteté, flou - [en effectuant des convolutions avec les filtres appris] (https://en.wikipedia.org/wiki/Kernel_ (traitement_image) ). Ces tâches ne sont pas effectuées sur l'image d'origine, mais sur la sortie du calque précédent. c'est pourquoi la sortie des couches les plus à droite devient très difficile à interpréter.

La tâche à effectuer dépend du filtre utilisé. Nous utilisons ici les filtres intégrés à VGG16, mais nous pourrions en principe les apprendre, si nous formions un CNN à partir de rien.

Un exercice intéressant, qui fournit un moyen d’interpréter la sortie des couches les plus à droite, consiste à trouver l’image [qui maximise l’activation d’un filtre donné] (https://blog.keras.io/how-convolutional-neural-networks-see-the-world.html).

Enfin, nous prenons la sortie du réseau VGG16 et la connectons au classifieur à 2 couches que nous avions auparavant.

In [ ]:
from keras.optimizers import rmsprop

model = Sequential()
model.add(Flatten(input_shape=train_features.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer=rmsprop(lr=2e-5),
              loss='binary_crossentropy',
              metrics=['acc'])

model.summary()

Entrainons-le !

In [ ]:
model.fit(
    train_features, train_labels,
    epochs = 20,
    batch_size = 16,
    validation_data = (valid_features, valid_labels)
)

In [ ]:
test_loss, test_acc = model.evaluate(valid_features, valid_labels)
print("accuracy on test set:", test_acc)

Ça a l'air bien non? Et certainement beaucoup plus vite :-)

In [ ]:
# Load 8 images at random, pass them through VGG16 and then through classifier
datagen = ImageDataGenerator(rescale=1./255)
generator = datagen.flow_from_directory("mldata/cats_and_dogs_small/valid",
                         target_size=(img_width, img_height),
                         batch_size=8,
                         class_mode="binary")
batch = generator.next()
features = vgg16.predict(batch[0])
probs = model.predict_proba(features)

# Show images together with probabilities
fig, axs = plt.subplots(2, 4, figsize=(16, 8))
for i in range(8):
    axs[i // 4, i % 4].imshow(batch[0][i])
    axs[i // 4, i % 4].set_title("prob. dog: %.2f" % (probs[i]))
    
    axs[i // 4, i % 4].get_xaxis().set_visible(False)
    axs[i // 4, i % 4].get_yaxis().set_visible(False)

## References

1. https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
2. https://gist.github.com/fchollet/0830affa1f7f19fd47b06d4cf89ed44d
3. https://gist.github.com/fchollet/f35fbc80e066a49d65f1688a7e99f069
4. https://github.com/abursuc/dldiy-practicals/blob/master/10_05_lesson1.ipynb
5. https://github.com/fastai/courses/blob/master/deeplearning1/nbs/dogs_cats_redux.ipynb
6. http://www.cs.toronto.edu/~frossard/post/vgg16/
7. https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks/